In [30]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import simplify_logic
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import _tree, export_text
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relunn import get_reduced_model, prune_features
from deep_logic.utils.sigmoidnn import prune_equal_fanin
from deep_logic import logic

results_dir = './results/mnist'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
concepts = [f'c{i}' for i in range(10)]
n_rep = 10
tot_epochs = 4001
prune_epochs = 2000
concepts

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [2]:
# MNIST problem
num_workers = 0
batch_size = 128
valid_size = 0.2
# Data augmentation for train data + conversion to tensor
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
   
])# Data augmentation for test data + conversion to tensor
test_transforms= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=test_transforms)

In [3]:
# Finding indices for validation set
num_train = len(train_data)
indices = list(range(num_train))
#Randomize indices
np.random.shuffle(indices)
split = int(np.floor(num_train*valid_size))
train_index, test_index = indices[split:], indices[:split]# Making samplers for training and validation batches
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(test_index)# Creating data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layers
        self.conv1 = torch.nn.Conv2d(1, 8, 3, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, 3, padding =1)
        # linear layers
        self.fc1 = torch.nn.Linear(784, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 64)
        self.fc4 = torch.nn.Linear(64, 10) 
        # dropout
        self.dropout = torch.nn.Dropout(p=0.2)
        # max pooling
        self.pool = torch.nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        # convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flattening the image
        x = x.view(-1, 7*7*16)
        # linear layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        return F.softmax(x, dim=1)
        
model = Net()
print(model)
model.cuda()

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [5]:
criterion = torch.nn.CrossEntropyLoss()
if os.path.isfile('trained_model.pt'):
    model.load_state_dict(torch.load('trained_model.pt'))

else:
    # epochs to train for
    epochs = 25
    set_seed(0)

    # tracks validation loss change after each epoch
    minimum_validation_loss = np.inf 

    #optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):

        train_loss = 0
        valid_loss = 0

        # training steps
        model.train()
        for batch_index, (data, target) in enumerate(train_loader):
            # moves tensors to GPU
            data, target = data.cuda(), target.cuda()
            # clears gradients
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            # loss in batch
            loss = criterion(output, target)
            # backward pass for loss gradient
            loss.backward()
            # update paremeters
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)

        # validation steps
        model.eval()
        for batch_index, (data, target) in enumerate(valid_loader):
            # moves tensors to GPU
            data, target = data.cuda(), target.cuda()
            # forward pass
            output = model(data)
            # loss in batch
            loss = criterion(output, target)
            # update validation loss
            valid_loss += loss.item()*data.size(0)

        # average loss calculations
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)

        # Display loss statistics
        print(f'Current Epoch: {epoch}\nTraining Loss: {round(train_loss, 6)}\nValidation Loss: {round(valid_loss, 6)}')

        # Saving model every time validation loss decreases
        if valid_loss <= minimum_validation_loss and epoch > 20:
            print(f'Validation loss decreased from {round(minimum_validation_loss, 6)} to {round(valid_loss, 6)}')
            torch.save(model.state_dict(), 'trained_model.pt')
            minimum_validation_loss = valid_loss
            print('Saving New Model')
    
    model.load_state_dict(torch.load('trained_model.pt'))

In [6]:
# tracking test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU
    data, target = data.cuda(), target.cuda()
    # forward pass
    output = model(data)
    # batch loss
    loss = criterion(output, target)
    # test loss update
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(10):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print(f'Test Loss: {round(test_loss, 6)}')

for i in range(10):
    if class_total[i] > 0:
        print(f'Test Accuracy of {classes[i]}: {round(100*class_correct[i]/class_total[i], 2)}%')
    else:
        print(f'Test Accuracy of {classes[i]}s: N/A (no training examples)')
        
        
print(f'Full Test Accuracy: {round(100. * np.sum(class_correct) / np.sum(class_total), 2)}% {np.sum(class_correct)} out of {np.sum(class_total)}')

Test Loss: 1.472841
Test Accuracy of 0: 98.68%
Test Accuracy of 1: 98.89%
Test Accuracy of 2: 100.0%
Test Accuracy of 3: 98.63%
Test Accuracy of 4: 100.0%
Test Accuracy of 5: 97.06%
Test Accuracy of 6: 98.39%
Test Accuracy of 7: 100.0%
Test Accuracy of 8: 100.0%
Test Accuracy of 9: 98.8%
Full Test Accuracy: 99.11% 783.0 out of 790.0


In [7]:
pred_c_train = []
true_c_train = []
model.eval()
for batch_index, (data, target) in enumerate(train_loader):
    # moves tensors to GPU
    data, target = data.cuda(), target.cuda()
    # forward pass
    output = model(data)
    pred = torch.argmax(output, 1)
    pred_c_train.append(pred)
    true_c_train.append(target)

In [8]:
pred_c_train = torch.cat(pred_c_train)
true_c_train = torch.cat(true_c_train)
pred_c_train.shape

torch.Size([48000])

In [9]:
y_train = (true_c_train % 2 == 1).to(torch.long)
y_train.shape

torch.Size([48000])

In [10]:
x_train = OneHotEncoder(sparse=False).fit_transform(pred_c_train.cpu().detach().numpy().reshape(-1, 1))
x_train = torch.FloatTensor(x_train)
x_train.shape

torch.Size([48000, 10])

In [11]:
pred_c_test = []
true_c_test = []
model.eval()
for batch_index, (data, target) in enumerate(test_loader):
    # moves tensors to GPU
    data, target = data.cuda(), target.cuda()
    # forward pass
    output = model(data)
    pred = torch.argmax(output, 1)
    pred_c_test.append(pred)
    true_c_test.append(target)

In [12]:
pred_c_test = torch.cat(pred_c_test)
true_c_test = torch.cat(true_c_test)
pred_c_test.shape

torch.Size([10000])

In [13]:
y_test = (true_c_test % 2 == 1).to(torch.long)
y_test.shape

torch.Size([10000])

In [14]:
x_test = OneHotEncoder(sparse=False).fit_transform(true_c_test.cpu().detach().numpy().reshape(-1, 1))
x_test = torch.FloatTensor(x_test)
x_test.shape

torch.Size([10000, 10])

In [15]:
def train_nn(x_train, y_train, need_pruning, seed, device, relu=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 100),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 30),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(30, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.01 * torch.norm(module.weight, 1)
                loss += 0.01 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

# General pruning

In [16]:
need_pruning = True
method = 'pruning'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    model = train_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device), 
                                                                       target_class=target_class,
                                                                       topk_explanations=10,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           target_class=target_class,
                                                                           topk_explanations=10,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9960
	 Epoch 3000: train accuracy: 0.9960
	 Epoch 3500: train accuracy: 0.9960
	 Epoch 4000: train accuracy: 0.9960
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "~c0 & ~c2 & ~c4 & ~c6 & ~c8" - Accuracy: 1.0000
	 Elapsed time 53.99287390708923
	 Class 0 - Global explanation: "c0 | c2 | c4 | c6 | c8" - Accuracy: 1.0000
	 Elapsed time 57.6913800239563
Seed [2/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9960
	 Epoch 3000: train accuracy: 0.9960
	 Epoch 3500: train accuracy: 0.9960
	 Epoch 4000: train accuracy: 0.9960
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "c1 | c3 |

In [17]:
results_pruning = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_pruning.to_csv(os.path.join(results_dir, 'results_pruning.csv'))
results_pruning

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,pruning,0,~c0 & ~c2 & ~c4 & ~c6 & ~c8,c0 | c2 | c4 | c6 | c8,1.0,1.0,1.0,53.992874,57.691380
1,pruning,1,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0,1.0,53.825268,55.067847
2,pruning,2,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0,1.0,58.975568,50.578591
3,pruning,3,~c0 & ~c2 & ~c4 & ~c6 & ~c8,c0 | c2 | c4 | c6 | c8,1.0,1.0,1.0,46.087959,44.425677
4,pruning,4,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0,1.0,58.797069,56.528682
5,pruning,5,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0,1.0,57.040676,51.349673
6,pruning,6,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0,1.0,53.862360,68.913783
7,pruning,7,~c0 & ~c2 & ~c4 & ~c6 & ~c8,c0 | c2 | c4 | c6 | c8,1.0,1.0,1.0,84.725517,84.127758
8,pruning,8,~c0 & ~c2 & ~c4 & ~c6 & ~c8,c0 | c2 | c4 | c6 | c8,1.0,1.0,1.0,94.526564,92.347148
9,pruning,9,~c0 & ~c2 & ~c4 & ~c6 & ~c8,c0 | c2 | c4 | c6 | c8,1.0,1.0,1.0,79.410599,70.128842


# LIME

In [18]:
# need_pruning = False
# method = 'lime'
# methods = []
# splits = []
# explanations = []
# explanations_inv = []
# model_accuracies = []
# explanation_accuracies = []
# explanation_accuracies_inv = []
# elapsed_times = []
# elapsed_times_inv = []
# for seed in range(n_rep):
#     print(f'Seed [{seed+1}/{n_rep}]')
    
#     model = train_nn(x_train, y_train, need_pruning, seed, device)
    
#     y_preds = model(x_test.to(device)).cpu().detach().numpy()
#     model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
#     print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
#     # positive class
#     target_class = 1
#     start = time.time()
#     global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
#                                                                        x_train.to(device), y_train.to(device), 
#                                                                        target_class=target_class,
#                                                                        method=method, device=device)
#     elapsed_time = time.time() - start
#     if global_explanation:
#         explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
#         explanation = logic.base.replace_names(global_explanation, concepts)
#     print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
#     print(f'\t Elapsed time {elapsed_time}')
        
#     # negative class
#     target_class = 0
#     start = time.time()
#     global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
#                                                                            x_train.to(device), y_train.to(device), 
#                                                                            target_class=target_class,
#                                                                            method=method, device=device)
#     elapsed_time_inv = time.time() - start
#     if global_explanation_inv:
#         explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
#         explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
#     print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
#     print(f'\t Elapsed time {elapsed_time_inv}')
    
#     methods.append(method)
#     splits.append(seed)
#     explanations.append(explanation)
#     explanations_inv.append(explanation_inv)
#     model_accuracies.append(model_accuracy)
#     explanation_accuracies.append(explanation_accuracy)
#     explanation_accuracies_inv.append(explanation_accuracy_inv)
#     elapsed_times.append(elapsed_time)
#     elapsed_times_inv.append(elapsed_time_inv)

In [19]:
# results_lime = pd.DataFrame({
#     'method': methods,
#     'split': splits,
#     'explanation': explanations,
#     'explanation_inv': explanations_inv,
#     'model_accuracy': model_accuracies,
#     'explanation_accuracy': explanation_accuracies,
#     'explanation_accuracy_inv': explanation_accuracies_inv,
#     'elapsed_time': elapsed_times,
#     'elapsed_time_inv': elapsed_times_inv,
# })
# results_lime.to_csv(os.path.join(results_dir, 'results_lime.csv'))
# results_lime

# Weights

In [20]:
need_pruning = False
method = 'weights'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    model = train_nn(x_train, y_train, need_pruning, seed, device, relu=True)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device),
                                                                       topk_explanations=10, 
                                                                       target_class=target_class,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           topk_explanations=10, 
                                                                           target_class=target_class,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9960
	 Epoch 3000: train accuracy: 0.9960
	 Epoch 3500: train accuracy: 0.9960
	 Epoch 4000: train accuracy: 0.9960
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "c1 | c3 | c5 | c7 | c9" - Accuracy: 1.0000
	 Elapsed time 71.51128125190735
	 Class 0 - Global explanation: "c4 | (~c1 & ~c3 & ~c5 & ~c7 & ~c9)" - Accuracy: 1.0000
	 Elapsed time 63.094101667404175
Seed [2/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9960
	 Epoch 3000: train accuracy: 0.9960
	 Epoch 3500: train accuracy: 0.9960
	 Epoch 4000: train accuracy: 0.9960
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "

In [21]:
results_weights = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_weights.to_csv(os.path.join(results_dir, 'results_weights.csv'))
results_weights

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,weights,0,c1 | c3 | c5 | c7 | c9,c4 | (~c1 & ~c3 & ~c5 & ~c7 & ~c9),1.0,1.0000,1.0000,71.511281,63.094102
1,weights,1,c1 | c3 | c5 | c7 | c9,~c1 & ~c3 & ~c5 & ~c7 & ~c9,1.0,1.0000,1.0000,80.853533,76.036408
2,weights,2,c1 | c3 | c5 | c7 | c9,(~c1 & ~c3 & ~c5 & ~c7 & ~c9) | (~c1 & ~c2 & ~...,1.0,1.0000,0.8099,72.000501,78.610895
3,weights,3,c5 | (~c0 & ~c2 & ~c4 & ~c6 & ~c8),c0 | c2 | c4 | c6 | c8,1.0,0.9108,1.0000,93.330645,79.732831
4,weights,4,c1 | c3 | c5 | c7 | c9,c6 | (~c1 & ~c3 & ~c5 & ~c7 & ~c9),1.0,1.0000,1.0000,80.391113,78.420050
5,weights,5,c3 | c9 | (~c0 & ~c2 & ~c4 & ~c6 & ~c8),c0 | c2 | c4 | c6 | c8,1.0,0.7981,1.0000,78.793919,77.957949
6,weights,6,c1 | c3 | c5 | c7 | c9,c0 | c4 | (~c1 & ~c3 & ~c5 & ~c7 & ~c9),1.0,1.0000,1.0000,79.902306,77.043643
7,weights,7,(~c0 & ~c2 & ~c4 & ~c6 & ~c8) | (~c2 & ~c4 & ~...,c0 | c2 | c4 | c6 | c8,1.0,0.4889,1.0000,80.108166,78.326705
8,weights,8,c1 | c3 | c5 | c7 | c9,~c4 | (~c1 & ~c3 & ~c5 & ~c7 & ~c9),1.0,1.0000,0.4926,81.186690,76.373570
9,weights,9,(~c3 & ~c6) | (~c0 & ~c2 & ~c4 & ~c6 & ~c8),c0 | c2 | c4 | c6 | c8,1.0,0.1968,1.0000,79.297514,78.457548


# Psi network

In [22]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [23]:
need_pruning = True
method = 'psi'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    # positive class
    target_class = 1
    model = train_psi_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds > 0.5)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    start = time.time()
    global_explanation = logic.generate_fol_explanations(model, device)[0]
    elapsed_time = time.time() - start
    explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
    explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    model = train_psi_nn(x_train, y_train.eq(target_class), need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.eq(target_class).cpu().detach().numpy(), y_preds > 0.5)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    start = time.time()
    global_explanation_inv = logic.generate_fol_explanations(model, device)[0]
    elapsed_time_inv = time.time() - start
    explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, 
                                                              target_class, x_test, y_test)
    explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.7977
	 Epoch 3000: train accuracy: 0.7977
	 Epoch 3500: train accuracy: 0.7977
	 Epoch 4000: train accuracy: 0.7977
	 Model's accuracy: 0.7988
	 Class 1 - Global explanation: "(c7 & ~c4)" - Accuracy: 0.5954
	 Elapsed time 0.05112743377685547
	 Epoch 0: train accuracy: 0.5076
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.8995
	 Epoch 3000: train accuracy: 0.8995
	 Epoch 3500: train accuracy: 0.8995
	 Epoch 4000: train accuracy: 0.8995
	 Model's accuracy: 0.8991
	 Class 0 - Global explanation: "(~c1 & ~c3 & ~c5 & ~c7)" - Accuracy: 0.8991
	 Elapsed time 0.03689980506896973
Seed [2/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train 

	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9058
	 Epoch 3000: train accuracy: 0.9058
	 Epoch 3500: train accuracy: 0.9058
	 Epoch 4000: train accuracy: 0.9058
	 Model's accuracy: 0.9108
	 Class 0 - Global explanation: "(~c1 & ~c3 & (c6 | ~c9) & (~c7 | ~c9))" - Accuracy: 0.5935
	 Elapsed time 0.03590559959411621
Seed [10/10]
	 Epoch 0: train accuracy: 0.4924
	 Epoch 500: train accuracy: 0.9960
	 Epoch 1000: train accuracy: 0.9960
	 Epoch 1500: train accuracy: 0.9960
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.4924
	 Epoch 3000: train accuracy: 0.8995
	 Epoch 3500: train accuracy: 0.8995
	 Epoch 4000: train accuracy: 0.8995
	 Model's accuracy: 0.9018
	 Class 1 - Global explanation: "(~c0 & ~c2 & ~c6 & ~c8)" - Accuracy: 0.9018
	 Elapsed time 0.04986715316772461
	 Epoch 0: train accuracy: 0.5076
	 Epoch 500: train accuracy: 0.9960


In [24]:
results_psi = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_psi.to_csv(os.path.join(results_dir, 'results_psi.csv'))
results_psi

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,psi,0,(c7 & ~c4),(~c1 & ~c3 & ~c5 & ~c7),0.8991,0.5954,0.8991,0.051127,0.036900
1,psi,1,(~c4 & (c1 | c3) & (c1 | ~c8)),(~c1 & (c0 | c6) & (c0 | ~c3) & (c4 | ~c5)),0.7963,0.4913,0.5818,0.049867,0.075797
2,psi,2,(~c8 & (c1 | ~c2) & (c3 | ~c0) & (~c2 | ~c6)),(~c1 & ~c9 & (c0 | ~c7)),0.8098,0.1956,0.5954,0.105714,0.047872
3,psi,3,(c7 | c9 | (~c4 & ~c8) | (c3 & ~c0 & ~c4)),(~c3 & (c2 | c4 | c8) & (c2 | c8 | ~c7)),0.8062,0.3983,0.5954,0.177527,0.079000
4,psi,4,(~c4 & ~c8 & (c7 | ~c6) & (c9 | ~c2)),((c0 & c8) | (c2 & c8) | (c0 & ~c1) | (c2 & ~c...,0.9018,0.4069,0.6964,0.100245,0.124665
5,psi,5,(~c0 & ~c2 & ~c6 & ~c8),(~c5 & (c4 | ~c9)),0.6827,0.9018,0.5935,0.052871,0.058842
6,psi,6,(c5 & ~c4),(c0 | c2 | c4 | c6),0.9026,0.5818,0.9026,0.036901,0.052129
7,psi,7,(~c2 & ~c8 & (c1 | ~c0) & (c9 | ~c4)),(c0 | c6 | (c8 & ~c7)),0.7986,0.3912,0.7986,0.088757,0.044879
8,psi,8,(~c0 | ~c6),(~c1 & ~c3 & (c6 | ~c9) & (~c7 | ~c9)),0.9108,0.2988,0.5935,0.030919,0.035906
9,psi,9,(~c0 & ~c2 & ~c6 & ~c8),(~c1 & ~c3 & (c2 | ~c9)),0.8080,0.9018,0.5935,0.049867,0.048870


# Decision tree

In [25]:
need_pruning = False
method = 'decision_tree'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    classifier = DecisionTreeClassifier(random_state=seed)
    classifier.fit(x_train.cpu().detach().numpy(), y_train.cpu().detach().numpy())
    y_preds = classifier.predict(x_test.cpu().detach().numpy())
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    target_class = 1
    start = time.time()
    explanation = tree_to_formula(classifier, concepts, target_class)
    elapsed_time = time.time() - start
    print(f'\t Class {target_class} - Global explanation: {explanation}')
    print(f'\t Elapsed time {elapsed_time}')
    
    target_class = 0
    start = time.time()
    explanation_inv = tree_to_formula(classifier, concepts, target_class)
    elapsed_time_inv = time.time() - start
    print(f'\t Class {target_class} - Global explanation: {explanation_inv}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(model_accuracy)
    explanation_accuracies_inv.append(model_accuracy)
    elapsed_times.append(0)
    elapsed_times_inv.append(0)

Seed [1/10]
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 > 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 > 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 > 0.50) | (c1 <= 0.50 & c7 > 0.50) | (c1 > 0.50)
	 Elapsed time 0.000997781753540039
	 Class 0 - Global explanation: (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 <= 0.50 & c4 <= 0.50 & c2 <= 0.50 & c0 <= 0.50 & c8 <= 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 <= 0.50 & c4 <= 0.50 & c2 <= 0.50 & c0 <= 0.50 & c8 > 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 <= 0.50 & c4 <= 0.50 & c2 <= 0.50 & c0 > 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 <= 0.50 & c4 <= 0.50 & c2 > 0.50) | (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 <= 0.50 & c4 > 0.50)
	 Elapsed time 0.0
Seed [2/10]
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: (c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= 0.50 & c5 > 0.50

In [26]:
results_tree = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_tree.to_csv(os.path.join(results_dir, 'results_tree.csv'))
results_tree

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,decision_tree,0,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
1,decision_tree,1,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
2,decision_tree,2,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
3,decision_tree,3,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
4,decision_tree,4,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
5,decision_tree,5,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
6,decision_tree,6,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
7,decision_tree,7,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
8,decision_tree,8,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0
9,decision_tree,9,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,(c1 <= 0.50 & c7 <= 0.50 & c3 <= 0.50 & c9 <= ...,1.0,1.0,1.0,0,0


# Summary

In [27]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# # lime
# df_mean = results_lime[cols].mean()
# df_sem = results_lime[cols].sem()
# df_mean.columns = mean_cols
# df_sem.columns = sem_cols
# summary_lime = pd.concat([df_mean, df_sem])
# summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning, 
#                      summary_lime, 
                     summary_weights, summary_psi, summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_accuracy_inv_mean,elapsed_time_mean,elapsed_time_inv_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_accuracy_inv_sem,elapsed_time_sem,elapsed_time_inv_sem
pruning,1.00000,1.00000,1.00000,64.124445,63.115938,0.000000,0.000000,0.000000,5.084594,4.900455
weights,1.00000,0.83946,0.93025,79.737567,76.405370,0.000000,0.088024,0.052169,1.877726,1.520392
psi,0.83159,0.51629,0.68498,0.074380,0.060486,0.022848,0.074542,0.041906,0.014140,0.008477
tree,1.00000,1.00000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))